In [12]:
import pandas as pd

df=pd.read_excel('../data/Food Delivery Time Prediction Case Study.xlsx')
df.drop(df[['ID','Delivery_person_ID','Type_of_order','Type_of_vehicle']],axis=1,inplace=True)

In [13]:
from geopy.distance import geodesic

def compute_distance(row):
    rest=(row['Restaurant_latitude'],row['Restaurant_longitude'])
    delv=(row['Delivery_location_latitude'],row['Delivery_location_longitude'])
    return geodesic(rest,delv).km
df['distance']=df.apply(compute_distance,axis=1)

In [14]:
df.drop(df[['Restaurant_longitude','Restaurant_latitude','Delivery_location_latitude','Delivery_location_longitude']],axis=1,inplace=True)

In [15]:
df=df[df['distance']<100]

In [16]:
df['speed']=df['distance']/df['Time_taken(min)']

In [23]:
df.head()

,Delivery_person_Age,Delivery_person_Ratings,Time_taken(min),distance,speed
0,1.310461,0.854368,24,-1.196305,0.125864
1,0.783465,-0.425272,33,1.861891,0.610416
2,-1.148853,-0.745182,26,-1.459036,0.059604
3,1.486127,0.214548,21,-0.347276,0.370214
4,0.432135,-0.105362,30,-0.628859,0.206597


In [24]:
df['Delivery_person_Ratings']=df['Delivery_person_Ratings'].clip(lower=2.5,upper=5)

In [27]:
from sklearn.preprocessing import StandardScaler

colms=['Delivery_person_Age','Delivery_person_Ratings','distance','speed']

scale=StandardScaler()

df[colms]=scale.fit_transform(df[colms])
df[colms].describe()

,Delivery_person_Age,Delivery_person_Ratings,distance,speed
count,4.516200e+04,45162.0,4.516200e+04,4.516200e+04
mean,-1.227190e-17,0.0,8.495927e-18,-5.695418e-17
std,1.000011e+00,0.0,1.000011e+00,1.000011e+00
min,-2.554176e+00,0.0,-1.474370e+00,-1.400382e+00
25%,-7.975226e-01,0.0,-9.045829e-01,-7.160667e-01
50%,-9.486130e-02,0.0,-9.198187e-02,-2.091501e-01
75%,7.834653e-01,0.0,7.039196e-01,4.619083e-01
max,3.594110e+00,0.0,2.004624e+00,6.407756e+00


In [29]:
df.to_csv('../data/cleaned.csv')